# Analyzing the first-level results

After running our GLM analysis, we have the estimated amplitude responses of each voxel to each image class. Along with Noah Benson's anatomical template / Bayesian model, we also have each voxel's visual area and location in the visual field (in terms of eccentricity and polar angle). By combining the information contained within them, along with the dataframe describing each stimulus class, we can construct our tuning curves.

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import itertools
import pyPyrTools.JBhelpers as jbh
import pyPyrTools as ppt
import scipy as sp
from matplotlib.colors import LinearSegmentedColormap 

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
behav_results = h5py.File('../data/raw_behavioral/2017-Nov-07_wl_subj042_sess0.hdf5')
#behav_results = h5py.File('../data/raw_behavioral/2017-Nov-07_wl_subj045_sess0.hdf5')
#behav_results = h5py.File('../data/raw_behavioral/2017-Oct-09_wl_subj001_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
stim_df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

# Array full of the actual stimuli
stim = np.load('../data/stimuli/unshuffled.npy')

# for this, we just want any run, since they all contain the same classes and we don't care about their order
design_df, _, _ = sfp.design_matrices.create_design_df(behav_results, stim_df, 1)
design_df = design_df.reset_index(drop=True).sort_values(by="class_idx")
design_df = design_df[['w_r', 'w_a', 'class_idx', 'alpha', 'res']].set_index('class_idx')

stim_df = stim_df.set_index(['w_r', 'w_a'])
stim_df['class_idx'] = design_df.reset_index().set_index(['w_r', 'w_a'])['class_idx']
stim_df = stim_df.reset_index()

In [ ]:
df = pd.read_csv('/home/billbrod/Desktop/wl_subj001_bootstrapped.csv')

In [ ]:
subject_name = 'wl_subj001'
benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20171007_prisma/MRI_first_level/results/stim_class/{}-{}.mgz" % subject_name
benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path, 'full', '/home/billbrod/Desktop/wl_subj001_bootstrapped.csv', vareas=[1])

In [ ]:
subject_name = 'wl_subj042'
benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20171107/MRI_first_level/stim_class/results/{}-{}.mgz" % subject_name
benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path, 'full', eccen_bin=True, vareas=[1,2], class_nums=[11, 1])

In [ ]:
subject_name = 'wl_subj045'
# benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
benson_path = "%s/%s/surf/{}.template_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_natme)
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20171107/MRI_first_level/stim_class/results/{}-{}.mgz" % subject_name
benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path, 'summary', vareas=[1,2], eccen_bin=False)

Here we see the different stimulus classes, as plotted in frequency space, colored by their superclass. These numbers are roughly log-spaced (doubling).

In [ ]:
with sns.axes_style('white'):
    g = sns.FacetGrid(df[df.voxel==0], hue='stimulus_superclass', size=5, aspect=1)
    g.ax.plot([-150, 200], [0,0],'k--', alpha=.5)
    g.ax.plot([0, 0], [-100, 250], 'k--', alpha=.5)
    g.map(sns.plt.scatter, 'w_a', 'w_r')
    g.add_legend()
    _=g.ax.axis('equal')
    g.ax.set_xlim([-150, 200])
    g.ax.set_ylim([-100, 250])

In [ ]:
def create_sin(size, w_x, w_y, origin=None):
    if origin is None:
        origin = ((size+1)/2., (size+1)/2.)
    x, y = np.meshgrid(np.array(range(1, size+1))-origin[0], 
                       np.array(range(1, size+1))-origin[1])
    return np.cos(x*w_x + y*w_y)

size = 1000
masked_stim = np.zeros((size, size))
masked_grad = np.zeros((size, size))
w_x = .05
w_y = .05
tmp_stim = create_sin(size, w_x, w_y)
dx = w_x * np.ones((size, size))
dy = w_y * np.ones((size, size))
for i in range(0, size+3, 99):
    for j in range(0, size+3, 99):
        loc_x, loc_y = i,j
        mask = sfp.utils.create_circle_mask(loc_x, loc_y, 48, size)
        masked_stim += mask*tmp_stim
        masked_grad += mask*calc_grad_sin(dx, dy, loc_x, loc_y)[0]
jbh.showIm([masked_stim, masked_grad], ncols=3)

In [ ]:
def calc_sf_maps(size, alpha, w_r, w_a, origin=None):
    if origin is None:
        origin = ((size+1)/2., (size+1)/2.)
    x, y = np.meshgrid(np.array(range(1, size+1))-origin[0], 
                       np.array(range(1, size+1))-origin[1])
    dy = (2*y*(w_r/np.pi)) / ((x**2 + y**2 + alpha**2)*np.log(2)) + w_a / ((1 + (y/x)**2) * x)
    dx = (2*x*(w_r/np.pi)) / ((x**2 + y**2 + alpha**2)*np.log(2)) - (w_a * y) / ((1 + (y/x)**2) * x**2)
    return dx, dy

def calc_grad_sin_constant(dx, dy, loc_x, loc_y, alpha=50, phase=0, origin=None):
    size = dx.shape[0]
    x, y = np.meshgrid(np.array(range(1, size+1))-loc_x, 
                       np.array(range(1, size+1))-loc_y)
    if origin is None:
        origin = ((size+1)/2., (size+1)/2.)
    x_orig, y_orig = np.meshgrid(np.array(range(1, size+1))-origin[0], 
                       np.array(range(1, size+1))-origin[1])
    w_x = dx[loc_y, loc_x]
    w_y = dy[loc_y, loc_x]
    local_phase = np.mod(w_x*x_orig[loc_y, loc_x] + w_y*y_orig[loc_y, loc_x] + phase, 2*np.pi)
    return np.cos(w_x*x + w_y*y + local_phase), local_phase

def calc_grad_sin_polar(dx, dy, loc_x, loc_y, w_r, w_a, alpha=50, phase=0, origin=None):
    size = dx.shape[0]
    x, y = np.meshgrid(np.array(range(1, size+1))-loc_x, 
                       np.array(range(1, size+1))-loc_y)
    if origin is None:
        origin = ((size+1)/2., (size+1)/2.)
    x_orig, y_orig = np.meshgrid(np.array(range(1, size+1))-origin[0], 
                       np.array(range(1, size+1))-origin[1])
    local_x = x_orig[loc_y, loc_x]
    local_y = y_orig[loc_y, loc_x]
    w_x = dx[loc_y, loc_x]
    w_y = dy[loc_y, loc_x]
    local_phase = np.mod((w_r/np.pi)*np.log2(local_x**2 + local_y**2+alpha**2)+ w_a*np.arctan2(local_y,local_x) + phase, 2*np.pi)
    if w_x==0 and w_y==0:
        return 0
    else:
        return np.cos(w_x*x + w_y*y + local_phase)

In [ ]:
idx=2

def create_sin(size, w_x, w_y, origin=None):
    if origin is None:
        origin = ((size+1)/2., (size+1)/2.)
    x, y = np.meshgrid(np.array(range(1, size+1))-origin[0], 
                       np.array(range(1, size+1))-origin[1])
    return np.cos(x*w_x + y*w_y)

size = 1080
phases = np.zeros((len(range(0, size+3, 99)), len(range(0, size+3, 99))))
masked_stim = np.zeros((size, size))
masked_grad = np.zeros((size, size))
for i, loc_x in enumerate(range(0, size+3, 99)):
    for j, loc_y in  enumerate(range(0, size+3, 99)):
        mask = sfp.utils.create_circle_mask(loc_x, loc_y, 48, size)
        x, y = np.meshgrid(np.array(range(1, size+1)) - loc_x,
                           np.array(range(1, size+1))- loc_y)
        w_x = dxs[idx][loc_y, loc_x]
        w_y = dys[idx][loc_y][loc_x]
        def opt_sin((x,y), phase):
            return (mask*np.cos(w_x * x + w_y * y + phase)).ravel()
        popt, pcov = sp.optimize.curve_fit(opt_sin, (x,y), (mask*tmp_stims[idx]).ravel(), bounds=(0, 2*np.pi))
        phases[i, j] = popt[0]
        masked_stim += mask * tmp_stims[idx]
        masked_grad += opt_sin((x,y), popt[0]).reshape((size, size))

jbh.showIm([masked_stim, masked_grad], ncols=2)

In [ ]:
idx=2
masked_stim = np.zeros((1080, 1080))
masked_grad = np.zeros((1080,1080))
masked_pol = np.zeros((1080,1080))
w_r = stim_df[stim_df.index==stim_idx[idx]].w_r.unique()[0]
w_a = stim_df[stim_df.index==stim_idx[idx]].w_a.unique()[0]
masks = np.zeros((1080, 1080))
for i, loc_x in enumerate(range(50, 1080, 99)):
    for j, loc_y in enumerate(range(50, 1080, 99)):
        mask = sfp.utils.create_circle_mask(loc_x, loc_y, 48, 1080)
        masks += mask
        masked_stim += mask*tmp_stims[idx]
        local_grad = calc_grad_sin_polar(dxs[idx], dys[idx], loc_x, loc_y, w_r, w_a)
        masked_grad += mask*local_grad
masked_grad[~masks.astype(bool)] -= 1
jbh.showIm([masked_stim, masked_grad], ncols=3)

In [ ]:
idx=0
masked_stim = np.zeros((1080, 1080))
masked_grad = np.zeros((1080,1080))
masked_pol = np.zeros((1080,1080))
w_r = stim_df[stim_df.index==stim_idx[idx]].w_r.unique()[0]
w_a = stim_df[stim_df.index==stim_idx[idx]].w_a.unique()[0]
masks = np.zeros((1080,1080))
for i in range(27, 1080, 55):
    for j in range(27, 1080, 55):
        loc_x, loc_y = i, j
        mask = sfp.utils.create_circle_mask(loc_x, loc_y, 27, 1080)
        masks += mask
        masked_stim += mask*tmp_stims[idx]
        #masked_pol += mask*calc_grad_sin_polar(mags[idx], direcs[idx], loc_x, loc_y)
        masked_grad += mask*calc_grad_sin_polar(dxs[idx], dys[idx], loc_x, loc_y, w_r, w_a)
masked_grad[~masks.astype(bool)] -= 1
jbh.showIm([masked_stim, masked_grad], ncols=3)
#jbh.showIm([tmp_stims[idx], gradient_sin, polar_sin], ncols=3)

In [ ]:
idx=1
masked_stim = np.zeros((1080, 1080))
masked_grad = np.zeros((1080,1080))
masked_pol = np.zeros((1080,1080))
w_r = stim_df[stim_df.index==stim_idx[idx]].w_r.unique()[0]
w_a = stim_df[stim_df.index==stim_idx[idx]].w_a.unique()[0]
masks = np.zeros((1080, 1080))
for i in range(50, 1080, 99):
    for j in range(50, 1080, 99):
        loc_x, loc_y = i, j
        mask = sfp.utils.create_circle_mask(loc_x, loc_y, 48, 1080)
        masks += mask
        masked_stim += mask*tmp_stims[idx]
        #masked_pol += mask*calc_grad_sin_polar(mags[idx], direcs[idx], loc_x, loc_y)
        masked_grad += mask*calc_grad_sin_polar(dxs[idx], dys[idx], loc_x, loc_y, w_r, w_a)
masked_grad[~masks.astype(bool)] -= 1
jbh.showIm([masked_stim, masked_grad], ncols=3)
#jbh.showIm([tmp_stims[idx], gradient_sin, polar_sin], ncols=3)

In [ ]:
mags = []
direcs = []
stim_idx = [stim_df[(stim_df.w_r==181)&(stim_df.w_a==0)].index.values[0]]
stim_idx.append(stim_df[(stim_df.w_r==0)&(stim_df.w_a==181)].index.values[0])
stim_idx.append(stim_df[(stim_df.w_r==128)&(stim_df.w_a==128)].index.values[0])
tmp_stims = []
dxs, dys= [], []
for i in range(3):
    a_stim = stim[stim_idx[i],:,:]
    tmp_stims.append(a_stim)
    w_r = stim_df[stim_df.index==stim_idx[i]].w_r.values[0]
    w_a = stim_df[stim_df.index==stim_idx[i]].w_a.values[0]
    alpha = stim_df[stim_df.index==stim_idx[i]].alpha.values[0]
    R = ppt.mkR(a_stim.shape)
    x, y = np.where(a_stim!=127)
    Rmin, Rmax = R[x,y].min(), R[x,y].max()
    dx, dy = calc_sf_maps(1080, alpha, w_r, w_a)
    dxs.append(dx)
    dys.append(dy)
    direc = np.arctan2(dy, dx)
    mag = np.sqrt(dx**2 + dy**2)
    mags.append(mag)
    direcs.append(direc)
    dx[R<Rmin] = 0
    dx[R>Rmax] = 0
    dy[R<Rmin] = 0
    dy[R>Rmax] = 0
    mag[R<Rmin] = 0
    mag[R>Rmax] = 0
    direc[R<Rmin] = 0
    direc[R>Rmax] = 0

In [ ]:
jbh.showIm([j for i in zip(tmp_stims[:2], mags[:2], direcs[:2]) for j in i], ncols=3)

In [ ]:
jbh.showIm([j for i in zip(tmp_stims[2:], mags[2:], direcs[2:]) for j in i], ncols=3)

In [ ]:
jbh.showIm([j for i in zip(tmp_stims[2:], mags[2:], direcs[2:]) for j in i], ncols=3)

In [ ]:
mags[1][540,:].max()

In [ ]:
mags[1][540,:].max()

In [ ]:
names = ['circular', 'radial', 'spiral']
sns.plt.figure(figsize=(15,5))
sns.plt.subplot(121)
for i in range(3):
    sns.plt.plot(np.array(range(1080))-540.5, mags[i][540,:], label=names[i])
sns.plt.legend()
# sns.plt.subplot(122)
# for i in range(3):
#     sns.plt.plot(np.array(range(1080))-540.5, direcs[i][540,:]/np.pi, label=names[i])
# sns.plt.ylabel('Fraction of $\pi$')
# sns.plt.legend()

In [ ]:
names = ['circular', 'radial', 'spiral']
sns.plt.figure(figsize=(15,5))
sns.plt.subplot(121)
sns.plt.plot(np.array(range(1080))-540.5, mags[0][540,:]/mags[0][540,:], label=names[0])
sns.plt.plot(np.array(range(1080))-540.5, mags[1][540,:]/mags[0][540,:], label=names[1])
sns.plt.plot(np.array(range(1080))-540.5, mags[2][540,:]/mags[0][540,:], label=names[2])
sns.plt.legend()
# sns.plt.subplot(122)
# for i in range(3):
#     sns.plt.plot(np.array(range(1080))-540.5, direcs[i][540,:]/np.pi, label=names[i])
# sns.plt.ylabel('Fraction of $\pi$')
# sns.plt.legend()

In [ ]:
Rmin, Rmax = sfp.first_level_analysis.find_ecc_range_in_degrees(stim[0,:,:], 12)
print("Inside radius of stimulus annulus: %.02f" % Rmin)
print("Outside radius of stimulus annulus: %.02f" % Rmax)

In [ ]:
tmp_df = df[df.stimulus_superclass=='circular']
g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=5,)
#g.map(sns.regplot, 'rounded_freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'rounded_freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes.flatten():
    ax.set_xscale('log', basex=2)
g.add_legend()
g.fig.savefig('wl_subj042-raw.svg')

In [ ]:
g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=5, col='stimulus_superclass', col_wrap=2,
                  col_order=['circular', 'radial', 'forward spiral', 'reverse spiral', 'mixtures'])
#g.map(sns.regplot, 'freq_space_distance', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_distance', 'amplitude_estimate')
g.map_dataframe(sfp.utils.scatter_ci_dist, 'freq_space_distance', 'amplitude_estimate')
for ax in g.axes:
    ax.set_xscale('log', basex=2)
g.add_legend()

If there were no scaling in the visual system, such that neurons at different places in the visual field were expected to have similar properties, the bottom would all line up well, and it doesn't!

In [ ]:
classes_of_interest = []
classes_of_interest.extend(df[(df.stimulus_superclass=='circular')&(df.rounded_freq_space_distance==11)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='radial')&(df.rounded_freq_space_distance==64)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='forward spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='reverse spiral')&(df.rounded_freq_space_distance==23)].stimulus_class.unique())
#classes_of_interest.append(df[df.stimulus_superclass=='mixtures'].stimulus_class.values[0])

stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

g = sns.FacetGrid(df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

# I know this goes from about -pi/2 to pi/2, in pi/12 steps
ticks = [(np.pi*(i-6)/12.) for i in range(13)]
labels = ['$\\frac{%s*\\pi}{12}$'%(i-6) for i in range(13)]

tmp_df = df[df.stimulus_superclass=='mixtures']
g = sns.FacetGrid(tmp_df, hue='eccen', palette='Reds', size=6)
g.map(sns.regplot, 'freq_space_angle', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_angle', 'amplitude_estimate')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_angle', 'amplitude_estimate_median', 'amplitude_estimate_std_error') 
_=g.ax.set_xticks(ticks)
_=g.ax.set_xticklabels(labels)
g.add_legend()

In [ ]:
angles = sorted(df.freq_space_angle.unique())
stim_idxs = []
for ang in angles:
    class_of_interest = df[(df.freq_space_angle==ang)&(df.rounded_freq_space_distance==32)].stimulus_class.unique()[0]
    stim_idxs.append(stim_df[stim_df.class_idx==class_of_interest].index[0])

#stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
tmp_df = pd.DataFrame(df.groupby(['eccen_bin', 'w_r', 'w_a']).modelmd.mean()).reset_index()

g = sns.FacetGrid(tmp_df, col='eccen_bin', col_wrap=4, size=5)
cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
g.map(sfp.utils.scatter_heat, 'w_a', 'w_r', 'amplitude_estimate_median', vmin=tmp_df['amplitude_estimate_median'].min(), 
      vmax=tmp_df['amplitude_estimate_median'].max())
sns.plt.colorbar(cax=cbar_ax)
g.fig.subplots_adjust(right=.9, top=.9)
g.fig.suptitle('Average response amplitude estimates at each point in frequency space')

In [ ]:
tmp_df = pd.pivot_table(df, 'amplitude_estimate_median', 'rounded_freq_space_distance', 'eccen_bin')
norm_df = tmp_df.copy()
for col in norm_df.columns:
    norm_df[col] = norm_df[col] / norm_df[col].max()

In [ ]:
fig = sns.heatmap(tmp_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Average response amplitude estimates')

In [ ]:
fig = sns.heatmap(norm_df, cmap='Reds')
fig.invert_yaxis()
fig.set_title('Normalized average response amplitude estimates')

In [ ]:
sns.distplot(df.R2.values)

# Create plots for first year talk

In [ ]:
# for this, we're only using circular results
tmp_df = df[df.stimulus_superclass=='circular']
# tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num']]

In [ ]:
tmper_df = tmp_df[tmp_df.freq_space_distance==6.]
tmper_df = tmper_df.groupby(['eccen', 'bootstrap_num'])['amplitude_estimate'].mean().unstack().reset_index()

In [ ]:
hyp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num'], var_name='Frequency')
hyp_df = pd.DataFrame(hyp_df.groupby(['Frequency', 'value', 'bootstrap_num'])['amplitude_estimate'].mean()).reset_index()

In [ ]:
print("freq_space_distance min: %.03f, max: %.03f" % (tmp_df.freq_space_distance.min(), tmp_df.freq_space_distance.max()))
print("Halfway in log space: %.03f" % np.floor(2**((np.log2(181.) + np.log2(6.))/2.)))
# because these are circular, freq_space_distance==w_r
#stim_idx = stim_df[(stim_df.w_r.isin([6, 32, 181]))&(stim_df.w_a==0)].index[::8]
stim_idx = stim_df[(stim_df.w_r.isin([6, 32, ]))&(stim_df.w_a==0)].index[::8]
stims = [stim[i] for i in stim_idx]

We actually want to plot windows of the stimuli instead of just sins

In [ ]:
max_degree_rad = 12
scale_factor = 10
mask = sfp.utils.create_circle_mask(750, 350, scale_factor* 1080/(2*2*max_degree_rad), 1080)
stim_windows = [mask * s + ~mask.astype(bool)*127 for s in stims]


In [ ]:
import warnings
def fit_log_norm_ci(x, y, ci_vals=[2.5, 97.5], **kwargs):
    """fit log norm to data and plot the result

    to be used with seaborn.FacetGrid.map_dataframe
    """
    data = kwargs.pop('data')
    color = kwargs.pop('color')
    lines = []
    # we want to collapse hemispheres -- eventually this should be moved earlier
    tmp = pd.DataFrame(data.groupby([x, 'bootstrap_num'])[y].mean()).reset_index()
    for boot in data.bootstrap_num.unique():
        plot_data = tmp.groupby(x)[[y, 'bootstrap_num']].apply(lambda x, j: x[x.bootstrap_num==j], boot)
        plot_idx = plot_data.index.get_level_values(x)
        plot_vals = plot_data[y].values
        try:
            popt, _ = sp.optimize.curve_fit(sfp.utils.log_norm_pdf, plot_idx, plot_vals)
        except RuntimeError:
            warnings.warn("For eccentricity %s and frequency space %s, bootstrap %d was not well fit by a log Gaussian and so is skipped" % (data['Eccentricity (degrees)'].unique()[0], data['Frequency'].unique()[0], boot))
        else:
            lines.append(sfp.utils.log_norm_pdf(plot_idx, *popt))
    lines = np.array(lines)
    lines_mean = lines.mean(0)
    cis = np.percentile(lines, ci_vals, 0)
    sns.plt.fill_between(plot_idx, cis[0], cis[1], facecolor=color, alpha=.2, **kwargs)
    sns.plt.plot(plot_idx, lines_mean, color=color, **kwargs)
    return lines

In [ ]:
def compare_hypotheses(df, axis_imgs=None, **kwargs):
    """
    axis_imgs should be a 2d list / array containing 2-tuples. each tuple should contain the relative position of each
        image and the corresponding image to put on an axis. the first dimension corresponds to the first axis, the second
        to the second axis
    """
    tmp_df = df[df.stimulus_superclass=='circular']
#    tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num', 'hemi']]
#    tmp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num', 'hemi'], var_name='Frequency')
    tmp_df = tmp_df[['eccen', 'amplitude_estimate', 'freq_space_distance', 'Local spatial frequency (cpd)', 'bootstrap_num',]]
    tmp_df = pd.melt(tmp_df, ['eccen', 'amplitude_estimate', 'bootstrap_num',], var_name='Frequency')
    tmp_df = tmp_df.rename(columns={'eccen': 'Eccentricity (degrees)'})

    g = sns.FacetGrid(tmp_df, hue='Eccentricity (degrees)', row='Frequency',  palette='Reds', size=5, aspect=2, sharex=False,
                      row_order=['Local spatial frequency (cpd)', 'freq_space_distance'])
    #g.map_dataframe(sfp.utils.plot_mean, 'value', 'amplitude_estimate')
    #g.map_dataframe(sfp.utils.scatter_ci_dist, 'value', 'amplitude_estimate', [50, 50])
    g.map_dataframe(fit_log_norm_ci, 'value', 'amplitude_estimate', ci_vals=[16, 84])
    g.map(sns.regplot, 'value', 'amplitude_estimate', x_estimator=np.mean, fit_reg=False, ci=0)
    sns.plt.subplots_adjust(hspace=.6)
    for i, ax in enumerate(g.axes.flatten()):
        if i==1:
            ax.set_title('Response as function of stimulus')
        elif i==0:
            ax.set_xlim([2**-3.5, 2**4])
            ax.set_title('Response as function of local spatial frequency (cycles / degree)')
        ax.set_xscale('log', basex=2)
        for pos, img in axis_imgs[i]:
            sfp.utils.add_img_to_xaxis(g.fig, ax, img, pos, vmin=0, vmax=255, size=.15)
        ax.xaxis.set_visible(False)
        ax.set_ylabel("Response amplitude estimate")
    #g.ax.set_xlim([2**-3, 2**3.5])
    g.add_legend()
    return g

In [ ]:
with sns.plotting_context('poster'), sns.axes_style('white'):
    g = compare_hypotheses(df, [zip([.025, .6], stim_windows), zip([.05, .6], stims)])
    g.savefig('SF-wl_subj045-results.svg')

In [ ]:
tmper_df = tmp_df[['Local spatial frequency (cpd)', 'amplitude_estimate', 'eccen']]

def fit_lnorm(data):
    popt, pcov = sp.optimize.curve_fit(sfp.utils.log_norm_pdf, data.index, data.values)
    return sfp.utils.log_norm_pdf(data.index, *popt), popt


bandwidth = []
peak = []
for n, g in tmper_df.groupby('eccen'):
    mn_data = g.groupby(['Local spatial frequency (cpd)'])['amplitude_estimate'].mean()
    fit_data, params = fit_lnorm(mn_data)
    peak.append([n, mn_data.index[np.argmax(fit_data.values)]])
    bandwidth.append([n, np.exp(params[2])])

peak = np.array([((int(i[0])+int(i[2]))/2., j) for i,j in peak])
bandwidth = np.array([((int(i[0])+int(i[2]))/2., j) for i,j in bandwidth])
print peak
print bandwidth

In [ ]:
sns.plt.scatter(bandwidth[:,0], bandwidth[:,1], c=sns.color_palette('Reds', 6), s=75)

In [ ]:
def hyperbola(x, a):
    b = 1.
    period = x*a
    period[x<b] = a*b
    return 1./period

def fit_hyperbola(peak):
    popt, pcov = sp.optimize.curve_fit(hyperbola, peak[:,0], peak[:,1])
    return popt

opt_a = fit_hyperbola(peak)

In [ ]:
peak_01 = peak
opt_a_01 = opt_a

In [ ]:
ecc = np.linspace(.01, 9, 50)
RF_scale_factor = 4.
V1_RF_size = np.concatenate([np.ones(len(ecc[ecc<.5]))/RF_scale_factor, np.linspace(1/RF_scale_factor, 4/RF_scale_factor, len(ecc[ecc>=.5]))])
#V2_RF_size = np.concatenate([2*np.ones(len(ecc[ecc<4])), np.linspace(2, 2.5, len(ecc[ecc>=4]))])

Olsson_peak = [2.75, 2.11, 1.76, 1.47,1.24, 1.06, .88, .77, .66, .60]
Olsson_ecc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

with sns.plotting_context('poster', font_scale=1), sns.axes_style('white'):
    # because this doesn't represent data, just intuition, we use this.
#    with plt.xkcd():
    x = np.linspace(.01, 9, 50)
    y = []
    fig, axes = sns.plt.subplots(1,1, figsize=(13,6))
    ax = axes
    # this gives intuitive plots, currently we want the possible hypotheses instead
#         for i in range(3):
#             y.append(10/(x+2)+i)
#             ax.plot(x, y[-1],  label='V%s'%(i+1), color=['r','g','b'][i])
    colors = sns.color_palette(n_colors=3)
    for i, (p, o) in enumerate([[peak_01, opt_a_01], [peak_42, opt_a_42], [peak_45, opt_a_45]]):
#    ax.plot(ecc, hyperbola(ecc, opt_a), '-', label='scaling')
#        ax.plot(ecc, hyperbola(ecc, o), '-', label='scaling', c= colors[i])
        ax.plot(ecc, hyperbola(ecc, o), '-', label='Subject %s'%(i+1), c= colors[i])
#    ax.plot(ecc, np.ones(len(ecc))*RF_scale_factor, '--', label='constant')
#        ax.set_ylim((0,8))
#    sns.plt.scatter(peak[:, 0], peak[:, 1], c=sns.color_palette('Reds', 6), s=75,)# label='This study')
        if i==1:
            sns.plt.scatter(p[:, 0]+.07, p[:, 1], c=colors[i], s=75,)# label='This study')
        else:
            sns.plt.scatter(p[:, 0], p[:, 1], c=colors[i], s=75,)# label='This study')
#    sns.plt.scatter(Olsson_ecc, Olsson_peak, s=75, c= sns.color_palette('Blues', 10), label='Olsson pilot')
    ax.set_xlabel("Receptive field center eccentricity (degrees)")
    ax.set_ylabel("Peak spatial frequency (cpd)")
    ax.set_title("")
    ax.set_ylim((.5, 4.5))
    ax.set_xlim((0, 11))

    sns.plt.legend(title="Subjects", loc='best')
    ax.figure.savefig('results-hypotheses.svg', bbox_inches='tight')

# Double-check design matrix

In order to use this, run everything except for the `sfp.experiment.run` statement first, which will create pictures, one from each class, in the order the `design_df` thinks they are presented. Then run the `sfp.experiment.run` block on a two-monitor setup so one monitor can display the experiment while you have this notebook open in the other. This will allow you to make sure the stimuli are being ordered correctly.

You can also compare `design_df.index.values` to a picture of the design matrix to make sure things got transferred correctly there as well.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import pyPyrTools.JBhelpers as jbh

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
behav_results = h5py.File('../data/raw_behavioral/2017-Aug-23_Noah_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
stim_df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

# Array full of the actual stimuli
stim = np.load('../data/stimuli/unshuffled.npy')

# for this, we just want any run, since they all contain the same classes and we don't care about their order
design_df, stim_length, TR = sfp.first_level_analysis.create_design_df(behav_results, stim_df, 1, drop_blanks=False)
design_df = design_df.reset_index(drop=True).set_index("class_idx")

stim_df['class_idx'] = np.floor(stim_df['index']/8)
stim_df = stim_df.set_index('class_idx')
stim_df['Onset time (TR)'] = design_df['Onset time (TR)']
stim_df = stim_df.reset_index().sort('Onset time (TR)')

stim_df = stim_df.drop_duplicates('class_idx')
stim_idx = stim_df.index.values

In [ ]:
data = sfp.experiment.run('../data/stimuli/unshuffled.npy', '../data/stimuli/Noah_run01_idx.npy', None, screen=0)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[:10]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[10:20]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[20:30]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[30:40]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[40:50]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[50:60]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[60:]], ncols=5, zoom=.2)